In [5]:
%pylab qt

[2023-11-30 12:01:14,418] {pyplot.py:414} DEBUG - Loaded backend QtAgg version 5.15.7.
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/jericho/anaconda3/envs/tigre_fastcat/lib/python3.11/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['f', 'det']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [6]:
from skimage.restoration import denoise_tv_chambolle

In [7]:
# %%
# Test the fastcat mhd file reader from fastcat.utils
import numpy as np
import fastcat as fc
import pickle 
#%%
pickle_file = '/home/jericho/1-Workspace/opengate_ggems_comparison/fastcat_scatter_test/test/ggems_1e03_121kVp.pkl'

# Load the fastcat simulation from the pickle file
with open(pickle_file, 'rb') as f:
    phantom = pickle.load(f)
    print('Done loading simulation parameters from ' + pickle_file)

# Get the simulation parameters
simulation_parameters = phantom.simulation_parameters
detector_material = simulation_parameters['detector_material']
spectrum = simulation_parameters['spectrum']

# Load the spectrum as a fastcat spectrum object
s = fc.Spectrum()
s.load(spectrum_file=spectrum)

det = fc.Detector(s,'CsI-784-micrometer')
angles = np.linspace(0,np.pi*2,2,endpoint=False) 

# return the projections
phantom.return_projs(det,s,angles,mgy=0, 
                    ASG=False, scat_on=False,
                    det_on=False,
                    filter_on=False,
                    return_intensity=True)


Done loading simulation parameters from /home/jericho/1-Workspace/opengate_ggems_comparison/fastcat_scatter_test/test/ggems_1e03_121kVp.pkl
[2023-11-30 12:01:30,210] {simulate.py:476} INFO -     Interp scatter 512 to 512 pixels
[2023-11-30 12:01:30,211] {simulate.py:476} INFO -     Interp scatter 512 to 512 pixels
[2023-11-30 12:01:31,051] {simulate.py:579} INFO - Running Simulations
[2023-11-30 12:01:31,052] {simulate.py:606} INFO -     Simulating 10 keV
[2023-11-30 12:01:31,452] {simulate.py:606} INFO -     Simulating 20 keV
[2023-11-30 12:01:31,856] {simulate.py:606} INFO -     Simulating 30 keV
[2023-11-30 12:01:32,257] {simulate.py:606} INFO -     Simulating 40 keV
[2023-11-30 12:01:32,652] {simulate.py:606} INFO -     Simulating 50 keV
[2023-11-30 12:01:33,039] {simulate.py:606} INFO -     Simulating 60 keV
[2023-11-30 12:01:33,439] {simulate.py:606} INFO -     Simulating 70 keV
[2023-11-30 12:01:33,839] {simulate.py:606} INFO -     Simulating 80 keV
[2023-11-30 12:01:34,250] {si

In [8]:

# kv_max = spectrum.x.max()

# %%
# Read in the first mhd file from the ggems_scatter simulation
mhd_file = '/home/jericho/1-Workspace/opengate_ggems_comparison/fastcat_scatter_test/test/out/ggems_1e09_121kVp_0000.mhd'
mhd_scatter_file = '/home/jericho/1-Workspace/opengate_ggems_comparison/fastcat_scatter_test/test/out/ggems_1e09_121kVp_0000-scatter.mhd'

fc_projection = phantom.intensity[0]

# image1_path = '/home/jericho/1-Workspace/opengate_ggems_comparison/catphan/out/ggems_1e09_121kVp_edepTrue_edep.mhd'
# fc_projection = fc.utils.read_mhd(mhd_file)

# ggems_primary = np.fliplr(image1)
# Read the mhd file from data
ggems_projection = fc.utils.read_mhd2(mhd_file)
ggems_projection_scatter = fc.utils.read_mhd2(mhd_scatter_file)

ggems_primary = ggems_projection - ggems_projection_scatter

# Crop the ggems and fastcat projection
edge = 1
ggems_primary = ggems_primary[:-edge,:]
fc_projection = fc_projection[:-edge,:]

# define the standard scaler function
def standard_scaler_ref(x, edge):
    return (x - x[edge:-edge,edge:-edge].mean()) / x[edge:-edge,edge:-edge].std(), x[edge:-edge,edge:-edge].mean(), x[edge:-edge,edge:-edge].std()

def rescale(x_new, mean, std, edge=10):
    # x_new = (x - x[edge:-edge,edge:-edge].mean()) / x[edge:-edge,edge:-edge].std()
    return x_new * std + mean


fc_projection, fc_mean, fc_std = standard_scaler_ref(fc_projection,edge=30)
ggems_rescale, ggems_mean, ggems_std = standard_scaler_ref(ggems_primary,edge=30)

phantom.fc_parameters = {}
phantom.fc_parameters['fc_mean'] = fc_mean
phantom.fc_parameters['fc_std'] = fc_std
phantom.fc_parameters['ggems_mean'] = ggems_mean
phantom.fc_parameters['ggems_std'] = ggems_std

# Rescale the fastcat projection to match the ggems projection
fc_projection = rescale(fc_projection, ggems_mean, ggems_std)

# Make a plot of the fc_projection and the ggems_primary projection
import matplotlib.pyplot as plt
plt.figure()
plt.plot(np.mean(fc_projection,axis=0))
plt.plot(np.mean(ggems_primary,axis=0))

In [10]:
import mpl_interactions as pli

In [12]:
# Now lets use these as a normalization factor for fastcat
import glob
# And lets interpolate the ggems scatter and denoise it as well
from scipy.interpolate import interpn

# Load all of the scatter files
ggems_scatter_files = glob.glob('/home/jericho/1-Workspace/opengate_ggems_comparison/fastcat_scatter_test/test/out/ggems_1e09_121kVp_*-scatter.mhd')
ggems_scatter_files.sort()

# Load the first scatter file
ggems_scatter = fc.utils.read_mhd2(ggems_scatter_files[0])

# Create an array to store the scatter projections
ggems_scatter_projections = np.zeros((len(ggems_scatter_files), ggems_scatter.shape[0], ggems_scatter.shape[1]))

# Loop through the scatter files and store them in the array
for i, file in enumerate(ggems_scatter_files):
    ggems_scatter = fc.utils.read_mhd2(file)
    ggems_scatter_projections[i,:,:] = ggems_scatter

# Visualize using hyperslicer
plt.figure()
controls = pli.hyperslicer(ggems_scatter_projections, axis=0,autoscale_cmap=False)
# denoise the ggems scatter


In [13]:
plt.figure()
plt.imshow(np.mean(ggems_scatter_projections,axis=0))

In [14]:
denoised = np.zeros_like(ggems_scatter_projections)

for ii, image in enumerate(ggems_scatter_projections):
    denoised[ii,:,:] = denoise_tv_chambolle(image, weight=4, eps=1e-5, max_num_iter=200)


KeyboardInterrupt: 

In [43]:
from skimage.restoration import denoise_tv_chambolle, denoise_tv_bregman

# Try a bunch of different weights
std_devs = []
hq_im = np.mean(ggems_scatter_projections,axis=0)
weights = np.linspace(0.1,4,20)
for weight in weights:

    denoised = denoise_tv_chambolle(image, weight=weight, max_num_iter=200)
    # append the rmse of the denoised image compared to the high quality image
    std_devs.append(np.sqrt(np.mean((denoised - hq_im)**2)))

plt.figure()
plt.plot(weights,std_devs)

# Could I do the same thing but also check what eps is best?
# Try a bunch of different weights
std_devs = []
hq_im = np.mean(ggems_scatter_projections,axis=0)
weights = np.linspace(0.1,4,10)
epsilons = np.linspace(1e-4,1e-2,10)
for ii, weight in enumerate(weights):
    std_devs.append([])
    for eps in epsilons:
        denoised = denoise_tv_bregman(image, weight=weight, eps=eps, max_num_iter=200)
        # append the rmse of the denoised image compared to the high quality image
        std_devs[ii].append(np.sqrt(np.mean((denoised - hq_im)**2)))

plt.figure()
plt.imshow(std_devs)

In [33]:
# Now lets try the same thing with gaussian denoising
from skimage.restoration import denoise_nl_means, estimate_sigma

# Try a bunch of different weights
std_devs = []
hq_im = np.mean(ggems_scatter_projections,axis=0)
weights = np.linspace(0.1,4,20)
# epsilons = np.linspace(1e-5,1e-3,20)
for ii, weight in enumerate(weights):

    denoised = denoise_nl_means(image, patch_size=5, patch_distance=3, h=weight*estimate_sigma(image), fast_mode=True)
    # append the rmse of the denoised image compared to the high quality image
    std_devs.append(np.sqrt(np.mean((denoised - hq_im)**2)))

In [35]:
plt.figure()
plt.imshow(denoised)

In [29]:
plt.figure()
pli.hyperslicer(denoised, axis=0,autoscale_cmap=False)

AttributeError: 'Controls' object has no attribute 'vbox'

In [41]:
import numpy as np
from matplotlib import pyplot as plt

from skimage.data import chelsea
from skimage.restoration import calibrate_denoiser, denoise_wavelet

from skimage.util import img_as_float, random_noise
from functools import partial

# rescale_sigma=True required to silence deprecation warnings
_denoise_wavelet = partial(denoise_wavelet, rescale_sigma=True)

noisy = img_as_float(image)
sigma = 0.3
# noisy = random_noise(image, var=sigma ** 2)

weights = np.linspace(0.1,4,10)
epsilons = np.linspace(1e-4,1e-2,10)

# Parameters to test when calibrating the denoising algorithm
parameter_ranges = {'sigma': np.arange(0.1, 0.3, 0.02),
                    'wavelet': ['db1', 'db2'],
                    'convert2ycbcr': [ False]}

# Denoised image using default parameters of `denoise_wavelet`
default_output = denoise_wavelet(noisy, rescale_sigma=False)

# Calibrate denoiser
calibrated_denoiser = calibrate_denoiser(noisy,
                                         _denoise_wavelet,
                                         denoise_parameters=parameter_ranges)

# Denoised image using calibrated denoiser
calibrated_output = calibrated_denoiser(noisy)

fig, axes = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(15, 5))

for ax, img, title in zip(
        axes,
        [noisy, default_output, calibrated_output],
        ['Noisy Image', 'Denoised (Default)', 'Denoised (Calibrated)']
):
    ax.imshow(img)
    ax.set_title(title)
    ax.set_yticks([])
    ax.set_xticks([])

plt.show()

In [40]:
plt.figure()
plt.imshow(noisy)

In [64]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec

from skimage.data import chelsea, hubble_deep_field
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.restoration import (calibrate_denoiser,
                                 denoise_wavelet,
                                 denoise_tv_chambolle, denoise_nl_means,
                                 estimate_sigma)
from skimage.util import img_as_float, random_noise
from skimage.color import rgb2gray
from functools import partial


from skimage.restoration.j_invariant import _invariant_denoise

image = hq_im
sigma = 0.4
noisy = ggems_scatter_projections[0] #random_noise(image, mode='speckle', var=sigma ** 2)

parameter_ranges_tv = {'weight': np.arange(0.01, 0.3, 0.02)}
_, (parameters_tested_tv, losses_tv) = calibrate_denoiser(
                                    noisy,
                                    denoise_tv_chambolle,
                                    denoise_parameters=parameter_ranges_tv,
                                    extra_output=True)
print(f'Minimum self-supervised loss TV: {np.min(losses_tv):.4f}')

best_parameters_tv = parameters_tested_tv[np.argmin(losses_tv)]
denoised_calibrated_tv = _invariant_denoise(noisy, denoise_tv_chambolle,
                                            denoiser_kwargs=best_parameters_tv)
denoised_default_tv = denoise_tv_chambolle(noisy, **best_parameters_tv)

psnr_calibrated_tv = psnr(image, denoised_calibrated_tv)
psnr_default_tv = psnr(image, denoised_default_tv)

parameter_ranges_wavelet = {'sigma': np.arange(0.01, 0.3, 0.03)}
_, (parameters_tested_wavelet, losses_wavelet) = calibrate_denoiser(
                                                noisy,
                                                _denoise_wavelet,
                                                parameter_ranges_wavelet,
                                                extra_output=True)
print(f'Minimum self-supervised loss wavelet: {np.min(losses_wavelet):.4f}')

best_parameters_wavelet = parameters_tested_wavelet[np.argmin(losses_wavelet)]
denoised_calibrated_wavelet = _invariant_denoise(
        noisy, _denoise_wavelet,
        denoiser_kwargs=best_parameters_wavelet)
denoised_default_wavelet = _denoise_wavelet(noisy, **best_parameters_wavelet)

psnr_calibrated_wavelet = psnr(image, denoised_calibrated_wavelet,data_range=10)
psnr_default_wavelet = psnr(image, denoised_default_wavelet,data_range=10)

sigma_est = estimate_sigma(noisy)

parameter_ranges_nl = {'sigma': np.arange(0.6, 1.4, 0.2) * sigma_est,
                       'h': np.arange(0.6, 1.2, 0.2) * sigma_est}

parameter_ranges_nl = {'sigma': np.arange(0.01, 0.3, 0.03)}
_, (parameters_tested_nl, losses_nl) = calibrate_denoiser(noisy,
                                                        denoise_nl_means,
                                                        parameter_ranges_nl,
                                                        extra_output=True)
print(f'Minimum self-supervised loss NL means: {np.min(losses_nl):.4f}')

best_parameters_nl = parameters_tested_nl[np.argmin(losses_nl)]
denoised_calibrated_nl = _invariant_denoise(noisy, denoise_nl_means,
                                            denoiser_kwargs=best_parameters_nl)
denoised_default_nl = denoise_nl_means(noisy, **best_parameters_nl)

psnr_calibrated_nl = psnr(image, denoised_calibrated_nl)
psnr_default_nl = psnr(image, denoised_default_nl)

print('                       PSNR')
print(f'NL means (Default)   : {psnr_default_nl:.1f}')
print(f'NL means (Calibrated): {psnr_calibrated_nl:.1f}')
print(f'Wavelet  (Default)   : {psnr_default_wavelet:.1f}')
print(f'Wavelet  (Calibrated): {psnr_calibrated_wavelet:.1f}')
print(f'TV norm  (Default)   : {psnr_default_tv:.1f}')
print(f'TV norm  (Calibrated): {psnr_calibrated_tv:.1f}')

plt.subplots(figsize=(10, 12))
plt.imshow(noisy, cmap='Greys_r')
plt.xticks([])
plt.yticks([])
plt.title('Noisy Image')

def get_inset(x):
    return x[0:100, -140:]

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15, 8))

for ax in axes.ravel():
    ax.set_xticks([])
    ax.set_yticks([])

axes[0, 0].imshow(get_inset(denoised_default_nl), cmap='Greys_r')
axes[0, 0].set_title('NL Means Default')
axes[1, 0].imshow(get_inset(denoised_calibrated_nl), cmap='Greys_r')
axes[1, 0].set_title('NL Means Calibrated')
axes[0, 1].imshow(get_inset(denoised_default_wavelet), cmap='Greys_r')
axes[0, 1].set_title('Wavelet Default')
axes[1, 1].imshow(get_inset(denoised_calibrated_wavelet), cmap='Greys_r')
axes[1, 1].set_title('Wavelet Calibrated')
axes[0, 2].imshow(get_inset(denoised_default_tv), cmap='Greys_r')
axes[0, 2].set_title('TV Norm Default')
axes[1, 2].imshow(get_inset(denoised_calibrated_tv), cmap='Greys_r')
axes[1, 2].set_title('TV Norm Calibrated')

for spine in axes[1, 2].spines.values():
    spine.set_edgecolor('red')
    spine.set_linewidth(5)

plt.show()


Minimum self-supervised loss TV: 21.2743


ValueError: image_true has intensity values outside the range expected for its data type. Please manually specify the data_range.

In [62]:
psnr(image, denoised_calibrated_tv, data_range=1)                      

-6.321150954965626

In [66]:
plt.figure()
plt.hist(ggems_scatter_projections[0].ravel(),bins=100)

(array([4.0000e+00, 0.0000e+00, 1.2000e+01, 0.0000e+00, 0.0000e+00,
        4.6000e+01, 0.0000e+00, 1.5500e+02, 0.0000e+00, 0.0000e+00,
        4.6200e+02, 0.0000e+00, 0.0000e+00, 1.0610e+03, 0.0000e+00,
        2.0670e+03, 0.0000e+00, 0.0000e+00, 3.8220e+03, 0.0000e+00,
        0.0000e+00, 6.5250e+03, 0.0000e+00, 9.7310e+03, 0.0000e+00,
        0.0000e+00, 1.3581e+04, 0.0000e+00, 1.7317e+04, 0.0000e+00,
        0.0000e+00, 2.0932e+04, 0.0000e+00, 0.0000e+00, 2.3384e+04,
        0.0000e+00, 2.4687e+04, 0.0000e+00, 0.0000e+00, 2.4799e+04,
        0.0000e+00, 0.0000e+00, 2.3248e+04, 0.0000e+00, 2.0496e+04,
        0.0000e+00, 0.0000e+00, 1.7812e+04, 0.0000e+00, 0.0000e+00,
        1.4655e+04, 0.0000e+00, 1.1340e+04, 0.0000e+00, 0.0000e+00,
        8.3090e+03, 0.0000e+00, 6.0590e+03, 0.0000e+00, 0.0000e+00,
        4.1810e+03, 0.0000e+00, 0.0000e+00, 2.7890e+03, 0.0000e+00,
        1.8770e+03, 0.0000e+00, 0.0000e+00, 1.1570e+03, 0.0000e+00,
        0.0000e+00, 7.0200e+02, 0.0000e+00, 4.09